In [1]:
from kedro.io import DataCatalog
from kedro.extras.datasets.pandas import CSVDataSet
import h5py
import glob
import pandas as pd

In [8]:
data_catalog = CSVDataSet(filepath="../data/01_raw/catalog.csv",
                        load_args={"sep": ';'})
io = DataCatalog(data_sets={'data_catalog': data_catalog})

df = io.load("data_catalog")
df

,cell_line,drug,file_path,note
0,D5,NaN,210310/D5/2301,NaN
1,PKP2-WT,NaN,220908/WT,NaN


In [3]:
path = df.loc[0,'file_path']
obj = h5py.File(r'/mnt/z/recordings/Mea1k/leejih/210310/D5/2301/highamp.raw.h5','r')

In [7]:
lsb = obj['settings']['lsb'][0] * 1e6
signals = (obj['sig']*lsb).astype('float32') 
signals.shape

512


: 

: 

In [ ]:
signals[0]

In [ ]:
# list all files in the directory
base_dir = "/mnt/z/recordings/Mea1k/leejih/"
rec_files = glob.glob(base_dir+df.loc[0,'file_path']+"/*.h5")

df2 = pd.DataFrame(columns=df.columns.to_list() + ['file_path_full'])
base_dir = "/mnt/z/recordings/Mea1k/leejih/"

for i in range(len(df)):
    rec_files = glob.glob(base_dir+df.loc[i,'file_path']+'/*.h5')
    tmp = pd.concat([df.loc[[i]]]*len(rec_files), ignore_index=True)
    tmp["file_path_full"] = rec_files
    df2 = df2.merge(tmp, how='outer')
df2

In [62]:
# Write a yaml file
import yaml

data = dict(n_files = 5)

def write_yaml(data):
    """A function to write YAML file."""
    with open('toyaml.yml', 'w') as f:
        yaml.dump(data, f)

write_yaml(data)